In [ ]:
# Libraries, Variables and Constants
import autogen, os
import json
from dotenv import load_dotenv # pip install python-dotenv

load_dotenv("./../credentials_my.env")
cache_seed = 42 # default seed is 41

In [ ]:
my_models = [
    {
        'model': os.environ['GPT-4T-20240409GA-128K'],
        "base_url": os.environ['AZURE_OPENAI_ENDPOINT_SEC'],
        'api_key': os.environ['AZURE_OPENAI_API_KEY_SEC'],
        "api_type": os.environ['OPENAI_API_TYPE'],
        "api_version": os.environ['AZURE_OPENAI_API_VERSION']
    },
    {
        'model': os.environ['GPT-4O-20240513-128K'],
        'api_key': os.environ['AZURE_OPENAI_API_KEY_SCUS'],
        "base_url": os.environ['AZURE_OPENAI_ENDPOINT_SCUS'],
        "api_type": os.environ['OPENAI_API_TYPE'],
        "api_version": os.environ['AZURE_OPENAI_API_VERSION']
    },
    {
        'model': os.environ['GPT-4T-VISIONPREVIEW-128K'],
        "base_url": os.environ['AZURE_OPENAI_ENDPOINT_SEC'],
        'api_key': os.environ['AZURE_OPENAI_API_KEY_SEC'],
        "api_type": os.environ['OPENAI_API_TYPE'],
        "api_version": os.environ['AZURE_OPENAI_API_VERSION']
    }
]

# need for converting to string
os.environ['models_var'] = json.dumps(my_models)

# Setting configurations for autogen
config_list = autogen.config_list_from_json(
    env_or_file='models_var',
    filter_dict={
        "model": {"gpt-4o-2024-05-13", "gpt-4"}
    }
)

# config_list

## Load autogen configuration from external JSON file

In [ ]:
# Setting configurations for autogen
config_list = autogen.config_list_from_json(
    env_or_file='models_list.json',
    filter_dict={
        "model": {"gpt-4o-2024-05-13", "gpt-4"}
    }
)

### [Difference between cache_seed and OpenAI's seed parameter](https://microsoft.github.io/autogen/docs/topics/llm-caching/)
OpenAI v1.1 introduced a new parameter `seed`. The difference between AutoGen's `cache_seed` and OpenAI's `seed` is AutoGen uses an explicit request cache to guarantee the exactly same output is produced for the same input and when cache is hit, no OpenAI API call will be made. OpenAI's `seed` is a best-effort deterministic sampling with no guarantee of determinism. When using OpenAI's `seed` with `cache_seed` set to `None`, even for the same input, an OpenAI API call will be made and there is no guarantee for getting exactly the same output.

In [ ]:
autogen.Completion.clear_cache()

llm_config={
    "cache_seed":  cache_seed, # None, cache_seed,  # seed for caching and reproducibility
    "config_list": config_list,  # a list of OpenAI API configurations
    "temperature": 0,  # temperature for sampling
}

# llm_config

In [ ]:
# create a UserProxyAgent instance named "user_proxy"

user_proxy = autogen.UserProxyAgent(
    name                       = "user_proxy",
    human_input_mode           = "NEVER", # NEVER / ALWAYS / TERMINATE
    max_consecutive_auto_reply = 10,

    # if the x["content"] ends by "TERMINATE", is_termination_msg-->True; otherwise, is_termination_msg--> False
    is_termination_msg         = lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    
    code_execution_config = {
        "work_dir": "coding",
        
        # Using docker is safer than running the generated code directly.
        # set use_docker=True if docker is available to run the generated code. 
        "use_docker": False
    },
    
    llm_config=llm_config
)

In [ ]:
# create an AssistantAgent named "assistant"

assistant = autogen.AssistantAgent(
    name       = "assistant",
    llm_config = llm_config
)

In [ ]:
%%time
# the assistant receives a message from the user_proxy, which contains the task description in the "message" field

chat_res = user_proxy.initiate_chat(
    assistant,
    message = """What date is today? Compare the year-to-date gain for META and TESLA.""",
    summary_method = "reflection_with_llm",
)

## Only if we couldn't gather the stock price

In [ ]:
%%time

# followup of the previous question, if needed
user_proxy.send(
    recipient=assistant,
    message="""Please try again and use another way to gather the stock price. For example, use yfinance.""",
)

## Checks the Steps

In [ ]:
i = 1
for cr in chat_res.chat_history:
    print(f'Step {i} --> {cr["content"].rstrip()[-9:]}')
    i=i+1

## Check the history in .cache/\<seed\>/cache.db

In [ ]:
# read the content of the cache

import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect(f'./.cache/{cache_seed}/cache.db')

# Create a cursor object
cur = conn.cursor()

# List all tables in the database
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(f"Tables in this DB: {cur.fetchall()}\n")

# Execute a query
cur.execute("SELECT * FROM Cache")

# Fetch and print the results
for row in cur.fetchall():
    print(row)

# Close the connection
conn.close()

## Ask an additional sub-question

In [ ]:
%%time

user_proxy.send(
    recipient=assistant,
    message="""
    Plot a chart of their stock prince change YTD. Save the data to stock_price_ytd.csv, and 
    save the plot to stock_price_ytd.png.
    """,
)

# Hic Sunt Leones